In [2]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager

In [3]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [4]:
#dataset을 선언합니다.
# dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
# dataset = dataset.repeat().batch(2)
# 분류 클래스는 3개이다.
nb_classes = 3 #class의 개수입니다.

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [10]:
# W and bias 셋팅
W = tfe.Variable(tf.random_normal([4,nb_classes]),name="weight")
b = tfe.Variable(tf.random_normal([nb_classes]),name="bias")
variables =[W,b]
print(variables)

[<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-1.2832785 , -0.57372373,  0.8555362 ],
       [ 0.36409333, -1.008571  , -1.1627557 ],
       [ 1.7711345 , -0.5514343 , -0.11697779],
       [ 0.9940506 ,  1.0928452 ,  1.4065602 ]], dtype=float32)>, <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 0.00882025, -0.35115844,  0.43264598], dtype=float32)>]


In [17]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
# 기존가설을 중 매트리스 연산으로 인해 XW+b값을 취함 또한 소프트맥스 함수 적용을 통해 확률값으로 변환하여 가설을 설정
def hypothsis(X):
    return tf.nn.softmax(tf.matmul(X,W)+b)
hypothsis(x_data)
# 소프트맥스를 통해나온 각로우의 합은 1이다.

<tf.Tensor: id=176, shape=(8, 3), dtype=float32, numpy=
array([[8.6973101e-01, 8.5733375e-03, 1.2169565e-01],
       [8.4027678e-01, 7.0474501e-04, 1.5901843e-01],
       [2.1421744e-01, 4.4507373e-04, 7.8533745e-01],
       [4.8144323e-01, 2.1568792e-05, 5.1853526e-01],
       [9.9999976e-01, 1.4124401e-09, 1.8492453e-07],
       [9.9942112e-01, 1.4904854e-06, 5.7730428e-04],
       [9.9999976e-01, 6.0302907e-10, 1.9464851e-07],
       [1.0000000e+00, 1.6535809e-11, 9.6672759e-09]], dtype=float32)>

In [19]:
# softmax onehot enocoding test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothsis(sample_db))

tf.Tensor([[6.5242233e-07 1.2419434e-06 9.9999809e-01]], shape=(1, 3), dtype=float32)


In [26]:
# 비용함수
def cost_fn(X,Y):
    logits = hypothsis(X)
    cost = -tf.reduce_sum(Y * tf.log(logits), axis=1) # axis = 열계산
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

In [24]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [27]:
# 경사하강법을 통해 파라메터를 최적화시킨다.
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: id=263, shape=(4, 3), dtype=float32, numpy=
array([[ 0.88976634, -0.74857426, -0.14119205],
       [ 1.5342798 , -1.2477098 , -0.28657004],
       [ 2.0546916 , -1.8734827 , -0.181209  ],
       [ 2.1013618 , -1.998515  , -0.10284694]], dtype=float32)>, <tf.Tensor: id=261, shape=(3,), dtype=float32, numpy=array([ 0.5506362 , -0.3737817 , -0.17685443], dtype=float32)>]


In [31]:
# 경사하강법을통해 W,b값을 갱신
# 2천번
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 0.061880
Loss at epoch 100: 0.060990
Loss at epoch 200: 0.060115
Loss at epoch 300: 0.059265
Loss at epoch 400: 0.058438
Loss at epoch 500: 0.057634
Loss at epoch 600: 0.056852
Loss at epoch 700: 0.056090
Loss at epoch 800: 0.055348
Loss at epoch 900: 0.054625
Loss at epoch 1000: 0.053920
Loss at epoch 1100: 0.053233
Loss at epoch 1200: 0.052563
Loss at epoch 1300: 0.051910
Loss at epoch 1400: 0.051273
Loss at epoch 1500: 0.050651
Loss at epoch 1600: 0.050043
Loss at epoch 1700: 0.049450
Loss at epoch 1800: 0.048871
Loss at epoch 1900: 0.048304
Loss at epoch 2000: 0.047751


In [35]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothsis(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[3.2850954e-04 2.3863547e-02 9.7580791e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)
